# 環境設定

In [ ]:
# 下載講師自製的 HappyML
import os

if not os.path.isdir("HappyML"):
  os.system("git clone https://github.com/cnchi/HappyML.git")

In [ ]:
# 載入必要套件
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from HappyML.pytorch import Sequential

In [ ]:
# 檢查是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


# 資料集前處理

In [ ]:
# 載入資料集
dataset = load_iris()
X, Y = dataset.data, dataset.target

# 切分訓練集、測試集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 特徵縮放
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 轉換成 PyTorch tensor
X_train, Y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(Y_train, dtype=torch.long)
X_test, Y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(Y_test, dtype=torch.long)

# 模型定義

In [ ]:
# 以快樂版建立模型，並嘗試丟入 GPU 中
model = Sequential().to(device)

# 定義模型的每個神經層、權重初始器、與激活函數
model.add(nn.Linear(4, 16), kernel_initializer="glorot_normal", activation="relu")
model.add(nn.Linear(16, 16), kernel_initializer="glorot_normal", activation="relu")
model.add(nn.Linear(16, 3), kernel_initializer="glorot_normal", activation="softmax")

# 加入優化器、損失函數，並將模型編譯好
model.compile(optimizer="adam", loss="categorical_crossentropy")

# 印出模型當前樣貌（非必要）
model.summary()

Sequential(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=3, bias=True)
    (5): Softmax(dim=-1)
  )
  (criterion): CrossEntropyLoss()
)


# 模型訓練

In [ ]:
# 訓練模型
model.fit(x=X_train, y=Y_train, validation_split=0.2, batch_size=32, epochs=500)

Epoch 1/500 - loss: 1.0305 - acc: 0.3125 - val_loss: 0.9902 - val_acc: 0.4167
Epoch 2/500 - loss: 1.0297 - acc: 0.3125 - val_loss: 0.9894 - val_acc: 0.4167
Epoch 3/500 - loss: 1.0290 - acc: 0.3125 - val_loss: 0.9886 - val_acc: 0.4167
Epoch 4/500 - loss: 1.0282 - acc: 0.3125 - val_loss: 0.9877 - val_acc: 0.4167
Epoch 5/500 - loss: 1.0274 - acc: 0.3125 - val_loss: 0.9869 - val_acc: 0.4167
Epoch 6/500 - loss: 1.0266 - acc: 0.3125 - val_loss: 0.9861 - val_acc: 0.4167
Epoch 7/500 - loss: 1.0258 - acc: 0.3125 - val_loss: 0.9852 - val_acc: 0.4167
Epoch 8/500 - loss: 1.0251 - acc: 0.3125 - val_loss: 0.9844 - val_acc: 0.4167
Epoch 9/500 - loss: 1.0244 - acc: 0.3125 - val_loss: 0.9836 - val_acc: 0.4167
Epoch 10/500 - loss: 1.0236 - acc: 0.3125 - val_loss: 0.9827 - val_acc: 0.4167
Epoch 11/500 - loss: 1.0228 - acc: 0.3125 - val_loss: 0.9819 - val_acc: 0.4167
Epoch 12/500 - loss: 1.0220 - acc: 0.3125 - val_loss: 0.9811 - val_acc: 0.4167
Epoch 13/500 - loss: 1.0213 - acc: 0.3125 - val_loss: 0.9803 

# 模型預測

In [ ]:
# 模型預測
Y_pred = model.predict(X_test)

df = pd.DataFrame({'Y_test': Y_test.numpy(), 'Y_pred': Y_pred.numpy()})
print(df)

    Y_test  Y_pred
0        1       1
1        0       0
2        2       2
3        1       2
4        1       2
5        0       0
6        1       1
7        2       2
8        1       2
9        1       1
10       2       2
11       0       0
12       0       0
13       0       0
14       0       0
15       1       2
16       2       2
17       1       1
18       1       1
19       2       2
20       0       0
21       2       2
22       0       0
23       2       2
24       2       2
25       2       2
26       2       2
27       2       2
28       0       0
29       0       0


# 模型評估

In [ ]:
# 模型評估
test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f"Loss of Testing Set: {test_loss:.4f}")
print(f"Accuracy of Testing Set: {test_acc:.4f}")

Loss of Testing Set: 0.7601
Accuracy of Testing Set: 0.8667
